In [1]:
import torch
import torch.nn as nn

from seisLM.data_pipeline import foreshock_aftershock_dataloaders as loaders
from seisLM.model.task_specific import foreshock_aftershock_models as models

dataloaders = loaders.prepare_foreshock_aftershock_dataloaders(
  num_classes=2,
  batch_size=32,
  event_split_method='temporal'
)

Seed set to 42
Seed set to 42


In [7]:
# Example usage
# Input shape: (batch_size, 3, 2500) where 3 is the number of channels
# and 2500 is the time length
model = models.Conv1DShockClassifier(
  in_channels=3,
  num_classes=2,
  num_layers=5,
  initial_filters=32,
  kernel_size=3,
)



In [8]:
from torchinfo import summary

summary(model, input_size=(32, 3, 2500))

Layer (type:depth-idx)                   Output Shape              Param #
Conv1DShockClassifier                    [32, 2]                   --
├─Sequential: 1-1                        [32, 512, 75]             --
│    └─DoubleConvBlock: 2-1              [32, 32, 1248]            --
│    │    └─Sequential: 3-1              [32, 32, 1248]            3,488
│    └─DoubleConvBlock: 2-2              [32, 64, 622]             --
│    │    └─Sequential: 3-2              [32, 64, 622]             18,688
│    └─DoubleConvBlock: 2-3              [32, 128, 309]            --
│    │    └─Sequential: 3-3              [32, 128, 309]            74,240
│    └─DoubleConvBlock: 2-4              [32, 256, 153]            --
│    │    └─Sequential: 3-4              [32, 256, 153]            295,936
│    └─DoubleConvBlock: 2-5              [32, 512, 75]             --
│    │    └─Sequential: 3-5              [32, 512, 75]             1,181,696
├─AdaptiveAvgPool1d: 1-2                 [32, 512, 1]         

In [4]:
train_loader = dataloaders['train']

batch = next(iter(train_loader))

In [5]:
out = model(batch[0].cuda())

In [6]:
out.shape


torch.Size([32, 2])